##### Задача:
1. Сгенерируйте датасет транзакций: (user_id, date, amount). Пусть будет 3 пользователя и по 5-6 транзакций у каждого в разные дни.
2. Найдите две последние транзакции для каждого пользователя.
3. Алгоритм:
 - Создать окно: partitionBy("user_id").orderBy(desc("date")).
 - Создать колонку rn = row_number().over(window).
 - Сделать фильтр rn <= 2.
 - Удалить вспомогательную колонку rn.

4. Сравните результат с исходным датасетом.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, FloatType
import datetime
import random

spark = SparkSession.builder.appName("Lab_04_02_TopN").getOrCreate()

# Определяем схему датафрейма
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("date", DateType(), True),
    StructField("amount", FloatType(), True)
])

# Данные для генерации
users = [1, 2, 3]
base_date = datetime.date(2026, 1, 1)

# Генерируем транзакции
transactions_data = []

for user_id in users:
    num_transactions = random.randint(5, 6)  # 5-6 транзакций на пользователя
    
    for i in range(num_transactions):
        # Генерируем случайную дату в пределах 30 дней от базовой даты
        days_offset = random.randint(0, 30)
        transaction_date = base_date + datetime.timedelta(days=days_offset)
        
        # Генерируем случайную сумму от 10 до 1000
        amount = round(random.uniform(10.0, 1000.0), 2)
        
        transactions_data.append((user_id, transaction_date, amount))

df = spark.createDataFrame(transactions_data, schema=schema)
df.show()

+-------+----------+------+
|user_id|      date|amount|
+-------+----------+------+
|      1|2026-01-03| 605.4|
|      1|2026-01-20|323.06|
|      1|2026-01-01|475.79|
|      1|2026-01-18|612.25|
|      1|2026-01-19|895.93|
|      2|2026-01-19| 805.0|
|      2|2026-01-12|602.45|
|      2|2026-01-09|349.56|
|      2|2026-01-02|579.55|
|      2|2026-01-19| 643.4|
|      3|2026-01-19| 82.36|
|      3|2026-01-07|418.69|
|      3|2026-01-19|591.31|
|      3|2026-01-24|104.39|
|      3|2026-01-22|734.15|
|      3|2026-01-05|363.95|
+-------+----------+------+



In [4]:
window_spec = Window.partitionBy(F.col("user_id")).orderBy(F.col("date").desc())

df_res = df.withColumn(
    "rn", F.row_number().over(window_spec)
).filter(F.col("rn") <= 2)

df_res.show()

+-------+----------+------+---+
|user_id|      date|amount| rn|
+-------+----------+------+---+
|      1|2026-01-20|323.06|  1|
|      1|2026-01-19|895.93|  2|
|      2|2026-01-19| 805.0|  1|
|      2|2026-01-19| 643.4|  2|
|      3|2026-01-24|104.39|  1|
|      3|2026-01-22|734.15|  2|
+-------+----------+------+---+

